# **DL-3. Свёрточные нейронные сети**

В этом модуле мы поговорим про свёрточные сети.

Задачи, которые мы научимся решать:

* задачи классификации (например, понять какой объект изображён на картинке);
* задачи регрессии (например, определить возраст человека на картинке).

Итак, приступим.

### Как компьютер воспринимает картинку?

Любая картинка состоит из пикселей, а в каждом пикселе закодирована яркость — это число от 0 до 255. Яркость 0 означает чёрный цвет, а значение 255 — белый. 

Для примера возьмём картинку и очень сильно её увеличим. После этого станут отчётливо видны пиксели, которые имеют различные яркости.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/274f963747811e05cd7694222eb5cc82/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3_sl_4.png)

В данном случае у нас чёрно-белая картинка, и пиксели выглядят чёрными, белыми и серыми. При работе с цветными изображениями всё становится несколько сложнее, потому что яркость будет определена для каждого из трёх цветовых каналов RGB (красного, синего, зелёного).

# 2. Как применять нейросеть

### Как применять нейросеть к картинке?

Давайте применим уже знакомый нам многослойный перцептрон. Пусть на вход у нас поступают пиксели картинки, которые образуют собой двумерную матрицу. Сопоставим каждой ячейке нашей пиксельной матрицы определённый вес, зависящий от яркости, а затем сложим все наши элементы матрицы с этими весами и проведём через сигмоиду, как мы это обычно делаем в простом нейроне.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/b6a2025ef1e38e79d988b38fcbeaeb08/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3_sl5.png)

При таком подходе могут возникнуть проблемы:

1. Картинка может быть очень большого размера. Допустим, если картинка имеет размеры 300 x 300, то нам нужно 90 000 весов (а это очень много).
Допустим, мы хотим определять, изображён ли на картинке котик.
1. Возьмём картинку, на которой котик изображён в правом нижнем углу. Во время градиентного спуска немного изменятся веса, которые смотрят непосредственно на этого котика (на этой картинке — красные веса).

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/6cb5a0f7b6b168720608d98fe9d1790f/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3_sl6_1.png)

Но если мы возьмём другую картинку, на которой котик будет находиться в левом верхнем углу, то у нас при градиентном спуске немного изменятся зелёные веса, потому что котик будет находиться в другой части картинки.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/669ea2bfe111af6396f3fa248247ff51/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3_sl6_2.png)

Таким образом, котик в разных частях картинки приводит к обновлению различных весов, хотя эти веса должны научиться находить на картинке одно и то же животное. То есть, если бы у нас была выборка картинок только с котиками в правом нижнем углу, мы бы никогда не научились находить котиков на картинках, на которых они расположены в левом верхнем углу.

# 3. Операция свёртки

Получается, что рассмотренный способ нам не подходит. Как же быть?

К счастью, для такого рода анализа изображений существует **операция свёртки**.

### Как работает операция свёртки?

Пусть на вход подаётся изображение, которое состоит из 0 и 1 (так нам будет удобнее устно считать и понимать, как именно работает свёртка), и пусть имеется скользящее окно фиксированного размера (в нашем случае 2 × 2). Нашим окошком 2 × 2 мы пробегаем по изображению и каждый кусочек, по которому пробегает окошко, поэлементно умножаем на некоторые веса и складываем. То есть получаем скалярное произведение нашего кусочка изображения и фильтра, состоящего из весов, которые мы будем обучать.

На картинке ниже представлен первый шаг, результат выполнения скалярного произведения кусочка изображения на фильтр — число 5.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/9914fbb1ef61e7300fdb018afcd2f9a2/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3_sl7.png)

Будем постепенно передвигать окошко и получать некие выходные данные. На картинке ниже показан один из шагов вычислений:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/c6b63aace6eec55cd2f9f78b54f71cb4/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3_sl8.png)

На самом деле, про свёртки известно очень давно. В любом редакторе, например, в Photoshop, есть реализации свёрток.

### Пример № 1

Пусть на вход подаётся картинка. Она пропускается через фильтр 3 × 3, у которого по центру стоит число 8, а во всех остальных ячейках –1 (сумма элементов равна 0).

Если этому фильтру подать на вход изображение, которое является однотонной заливкой, то, поэлементно умножив все яркости на коэффициенты и сложив, мы получим 0, который соответствует чёрному цвету. Те места, где заливка неоднотонная (есть перепад яркости), будут восприниматься как граница, и значение в этих ячейках будет отлично от 0. 

В результате свёртки получаем картинку с подсвеченными краями.

### Пример № 2

Рассмотрим другой фильтр, сумма весов которого равна 1. При однотонной заливке этот фильтр **не будет** менять цвет. Если заливка не будет однотонной, то на границе объекта на изображении у нас будет повышаться яркость. Визуально это будет восприниматься как увеличение резкости. Существует фильтр и для обратной операции — размытия. Такой фильтр состоит из одинаковых элементов и усредняет все цвета, которые видит.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/98f612957f50420c1a46c35e38f9cc07/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3_sl11.png)

# 4. Простой свёрточный слой

### Как реализовать простой свёрточный слой в нейросети?

Для начала нужно разобраться, как это поможет нам искать на картинках определённые объекты (кошек, собак, уток).

Рассмотрим пример. Пусть на картинке у нас расположена чёрточка под углом, и нам нужно научиться находить её на картинке. Тогда возьмём в качестве фильтра эту же чёрточку.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/592d0553247d6f547d28e8ee5fbd075e/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3_sl13.png)

Заметим, что после применения операции ненулевые значения на выходе будут только для двух положений скользящего окна (положения обозначены красной рамкой).

Посмотрим, как будет работать свёртка с картинкой, на которой чёрточка повёрнута в другую сторону. Применив операцию свёртки, получим немного другие значения: здесь тоже только два ненулевых значения на выходе, но они меньше, чем в прошлый раз.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/b700b8deede6c9137bcabc8ed8ce125e/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl14.png)

Это говорит нам о том, что на самом деле мы научились определять, что чёрточка на картинке повёрнута.

### Как же теперь нам понять, куда именно повернута чёрточка? 

Нужно просто взять максимум функции — именно он и будет определять поворот нашей чёрточки.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/724eb86b31be32cda01a2e021db14137/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl15.png)

Максимум функции — простой классификатор для изображений.

Данный классификатор обладает некоторыми полезными свойствами. Например, свёртка и сдвиг коммутативны (если эти операции поменять местами, то результат не изменится).

Посмотрим на примере, что произойдет на выходе, если чёрточка на изображении переместится в другой угол.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/61fa791fc0a460ed52ad8883c97946e7/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl18.png)

Снова получим для обоих случаев лишь два ненулевых положения окна. Причём для обоих случаев результат одинаков (только сдвинут). Это означает, что неважно, что делать вначале, сдвиг или свёртку — результат всё равно будет одинаков.

Это важный и полезный результат, так как наш классификатор — максимум одинаков. То есть неважно, где будет расположен котик на картинке (в левом верхнем или в правом нижнем углу), мы всё равно будем знать, что это котик.

Теперь мы можем собрать свёрточный слой в нейросети.

### Как это работает?

На вход подаётся зелёная картинка 3 × 3 с некой рамкой по бокам (обычно она заполнена нулями), называемой padding, или добавкой, необходимой для того, чтобы мы смогли поместить столько окошек, какого размера картинка, чтобы картинка осталась такого же размера за счёт этой добавки.

У нас есть веса, которые нейросеть будет обучать с помощью градиентного спуска. На картинке ниже показано, как будет посчитан первый нейрон.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/c2e65e752ec1aa35969f1cabef614902/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl19.png)

Кроме того, у нас есть ещё один параметр —  шаг (сдвиг, stride), с которым двигается наше скользящее окно. Здесь шаг — 1 пиксель.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/b053e6aadca17975dbd833db692bc73d/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl20.png)

Продолжаем так дальше и на выходе получаем также картинку 3 × 3 — результат свёртки, называемый feature map (карта фичей).

Заметим, что для этого преобразования мы использовали всего 10 параметров (9 весов и 1 шаг).

### Как работает градиентный спуск для операции свёртки?

На самом деле свёрточный слой — частный случай полносвязного. Веса, находящиеся вне нашего поля обзора, просто занулены. И тогда имеем полносвязный слой, а для полносвязного слоя мы уже умеем считать производную.

### Как считать производную для свёрточного слоя?

Возьмём картинку 3 × 3 и фильтр 2 × 2. На этой картинке есть всего четыре возможных положения фильтра. Последим за одним параметром w4. Нам интересно взять производную нашей функции потерь именно по этому параметру. Все использования нашего параметра в сети назовём различными буквами (a, b, c, d) и будем считать по ним производную.

Чтобы сделать шаг по градиенту, необходимо будет посчитать производную потери по каждой из этих букв. Сдвинем каждый параметр по направлению антиградиента, но если вспомнить что a, b, c, d — это не разные параметры, а один и тот же, то становится понятно, как на самом деле работает градиентный спуск для свёртки: мы четыре раза обновили один и тот же параметр и обновили его на сумму градиентов по всем использованиям на нашей картинке.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/4232c31fc316b416470f80b26506c23a/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl21.png)

Ещё одним плюсом свёрточного слоя является то, что для его реализации нужно очень мало параметров. Пусть у нас имеется картинка 300 × 300, и на выходе мы хотим получить картинку такого же размера. В случае с полносвязным слоем нам необходимо 8.1 * 109 параметров, в то время как в случае со свёрточным слоем и окном 5 × 5 — всего 25 параметров.

# 5. Практика. Простой свёрточный слой

Используемый в видео [ноутбук](https://colab.research.google.com/drive/1Wteyx4m3mbwnzkXBFnLRQo7Fwmc_Kl51).

В данной практике мы посмотрим, как в Keras настроить свёрточную сеть, а точнее — как добавить свёрточный слой в нашу сеть. Мы будем использовать уже известную нам задачу Fashion MNIST. 

Загрузим необходимые данные и будем собирать свёрточную сеть, добавляя в неё свёрточный слой Conv2D. Изучим параметры слоя и установим необходимые:

```py
model.add(L.Conv2D(1, kernel_size=3, strides=1, padding='same', input_shape=(28, 28, 1)))

#  6. Усложняем свёрточный слой

Теперь мы готовы усложнить свёрточный слой и применить его к какой-нибудь цветной картинке.

Картинку будем описывать тензором размера $W$ х $H$ х $С_{in} $, где $С_{in}$  — количество входных каналов (три канала RGB).

Для операции свёртки будем теперь «вырезать» не квадратик, как раньше, а кубик, чтобы отличать, например, рыжих котов от чёрных. Фильтр теперь тоже тензор, и его глубина также равна $С_{in}$. 

Независимо от того, какой размерности у нас картинка и фильтр, на выходе всё равно будет скаляр, так как мы берём входной кусочек объёма и объёмный фильтр, перемножаем и складываем, получая скаляр. То есть, применив свёртку к картинке, получим плоскую картинку.

Мы потеряли достаточно много информации, ведь на вход подавалась трёхмерная картинка, а на выходе получилась двумерная.  Тут-то и становится понятно, что одного фильтра мало.

### Как решить эту проблему?

Мы можем обучить много разных фильтров, все их применить, а результирующие картинки соединить в одну объёмную картинку на выходе. Каждый разрез этой объёмной выходной картинки будет хранить некую карту фичей, которую мы насчитаем.

Допустим, зелёный фильтр на картинке будет определять пиксели на горизонтальной границе, то есть всегда, когда мы будем на горизонтальной границе изображения, этот нейрон будет принимать какое-то большое значение.

Красный же фильтр научим определять пиксели на границе объекта по диагонали. И теперь в одном пикселе входного изображения появляется целый вектор признаков, которые мы можем использовать для дальнейшего анализа. Этих признаков мы можем сделать сколько угодно. Это наш гиперпараметр $С_{out}$  — количество выходных каналов в нашем объёме. Очевидно, что $С_{out} > 1$, а на практике может быть такое значение как 32 или 64.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/6a4ba8a3c39043f7c242ac674b691134/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl26.png)

Теперь у пикселя появляется глубина, и мы переходим в новые координаты. Исходные координаты RGB были неудобны, так как в них ничего непонятно было о картинке, а теперь мы можем перейти, например, в 32 признака, каждый из которых решает свою определённую подзадачу. Мы сможем понять, на какой части картинки мы изображены: на горизонтальной или на диагональной. 

Следующие фильтры, которые будут применяться, смогут учитывать все эти признаки одновременно и научатся решать более сложную задачу.

Вновь подчеркнём, как мало параметров мы использовали. Если наш фильтр имеет размеры $W_k * H_k * С_{in}$ , то для того чтобы научить Сout таких фильтров, необходимо всего лишь $(W_k * H_k * С_{in}) * С_{out}$ параметров.

### Да, и одного свёрточного слоя мало!

Допустим, на картинке 300 × 300 изображён кот, а наш первый свёрточный слой посмотрел на кусочки изображения 3 × 3. Эти кусочки слишком маленькие, мы не можем из них собрать кота.

Давайте посмотрим. что произойдёт с нашей картинкой и преобразованиями в нейросети, когда мы к первому результату нашей свёртки применим ещё один свёрточный слой.

На примере ниже — вход размера 5 × 5. Мы применили свёртку размера 3 × 3 и получили девять различных значений. Заметьте, что на первом свёрточном слое каждый нейрон (например, зелёный, выделенный на картинке) смотрит на кусочек изображения размером 3 × 3. Но если к результату первого свёрточного слоя ещё раз применить свёртку с какими-то другими значениями в фильтре того же размера 3 × 3, то любой нейрон на выходе второго слоя свёртки уже смотрит эффективно на кусочек картинки размером 5 × 5, так как все нейроны, с которыми слой работает, смотрят на целый кусочек размера 3 × 3. Если провести пунктирные линии, то видно, что у нашего нейрона «повысилось поле обзора», и он смотрит на больший кусочек изображения. То есть, создавая больше слоёв, мы имеем возможность определить котика всё большего и большего размера.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/96b159cba2665398de550d6c74ac6917/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl27.png)

### Что делать, если у нас котик занимает весь размер картинки (300 × 300)?

Посчитаем, сколько нам потребуется слоёв, чтобы распознать такого котика.

Рассмотрим это на простом одномерном примере. Допустим, у нас на входе одномерный вектор из девяти нейронов. Посмотрим, как будет выглядеть результат первой свёртки размером 3 × 1. 

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/b1175e91994b1bfd73c0a92e1a7b798a/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl28.png)

Каждый нейрон на первом свёрточном слое смотрит на три нейрона нашего изображения. Если мы применим ещё один свёрточный слой, то нейроны второго слоя будут смотреть уже на пять пикселей нашего изображения. Если так продолжать делать, то можно вывести формулу. Видно, что наше поле обзора прирастает линейно по количеству слоёв. Значит, чтобы посмотреть на котика размером 300 × 300, нам понадобится 150 слоёв, а это очень много. Возникает проблема, потому что столько слоёв будут очень долго просчитываться. Чуть позже мы научимся её решать.

# 7. Практика. Усложняем свёрточный слой

Используемый в видео [ноутбук](https://colab.research.google.com/drive/1Wteyx4m3mbwnzkXBFnLRQo7Fwmc_Kl51).

Теперь мы знаем, что свёрточный слой должен быть не один, и в нём должно быть больше фильтров, потому что, используя в свёрточном слое всего один фильтр, мы теряем информацию.

Все признаки пикселей мы можем закодировать в глубине, которую даёт нам свёртка.

Используя один слой и один канал на входе и на выходе, мы получали на выходе следующий объём: (None, 28, 28, 1)  

Попробуем обучить **десять фильтров**:

```py
model.add(L.Conv2D(filters=10, kernel_size=3, strides=1, padding='same', input_shape=(28, 28, 1)))
```

Заметим, что на выходе теперь десять каналов: (None, 28, 28, 10). Это обусловлено тем, что в каждом пикселе появилась глубина. Количество параметров = 100, по десять параметров на каждый фильтр. Соответственно, если у нас 32 фильтра, то получим 320 параметров.

Мы знаем, что одного свёрточного слоя мало, потому что сейчас каждый из нейронов смотрит на кусочек картинки 3 × 3, что объективно мало. 
**Добавим ещё один свёрточный слой:**

```py
model.add(L.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', input_shape=(28, 28, 1)))
model.add(L.Conv2D(filters=32, kernel_size=3, strides=1, padding='same'))
```
Во втором свёрточном слое получим уже 9 248 параметров! Откуда такое число? Давайте разберёмся.

Второй слой будет сворачивать все каналы входного для него изображения, работая с выходом из первого свёрточного слоя, то есть Keras будет вырезать кусочки размера 3 × 3 × 32. Таким образом, для одного фильтра потребуется 3 × 3 × 32 + 1 параметров. Всего на 32 фильтра: (3 × 3 × 32 + 1) * 32 =9 248.

Можно продолжать добавлять свёрточные слои, увеличивая количество каналов в них. Тогда слои будут использовать всё больше и больше параметров.

# 8. Пулинг слой

### Нужно растить поле обзора быстрее!

Самый простой способ увеличить поле обзора наших нейронов — увеличить шаг свёртки.

Возьмём шаг нашего скользящего окна равным двум. Тогда наша картинка станет меньше вдвое (то есть, если наша картинка была размером 300 × 300, то после свёртки с шагом два она стала 150 × 150). При добавлении новых свёрточных слоёв наша картинка каждый раз будет уменьшаться вдвое, а значит, чтобы найти кота размером на всю картинку. нам понадобится всего девять слоев (а не 150, как раньше).

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/90e052e54a556ae87700603f92d3921e/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl30.png)

Вспомним хорошее свойство нашего классификатора: после применения свёртки мы брали максимум. Когда мы берём свёртку, мы считаем пространственную корреляцию нашего кусочка изображения с тем шаблоном, который мы хотим на нём найти. Там, где корреляция большая, мы нашли что-то похожее на наш шаблон, а затем мы брали максимум и понимали, что наша чёрточка находится под разным углом.

В нашем свёрточном слое мы нигде не использовали функцию максимума, но нужно обязательно её использовать, чтобы была инвариантность к сдвигу изображения. В этом нам поможет **пулинг слой**. 

### Как работает пулинг слой?

На входе есть картинка (4 × 4), и есть окно фиксированного размера (2 × 2). Если мы проходим этим окном по картинке с шагом, равным двум, то у окна есть всего четыре возможных положения. Теперь пулинг слой применяет некоторую операцию (мы возьмём максимум). В оранжевом окошке максимум пять, и поэтому на выходе получаем пять, в фиолетовом — шесть и так далее.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/46ae07271d2eb700209ff025693d0ca6/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl32.png)

Пулинг слой комбинирует в себе два плюса:

* Он применяет максимум, а значит, имеет небольшую инвариантность к сдвигу.
* Пулинг слой обычно применяют с шагом = 2, а это значит, что наша картинка уменьшается в два раза, и поле обзора растёт быстрее.

Пулинг с максимумом применяется к различным feature map независимо от объёма, который мы получили: он работает на входе и на выходе с плоскими картинками и не меняет количество фильтров.

Пулинг с максимумом теряет детали изображения (на картинке выше показан результат свёртки 200 × 200, к которому применяют пулинг и в итоге получают картинку 100 × 100).

### Как считать градиент для пулинга

Возникает ещё один важный вопрос: как считать градиент для пулинга, если операция максимума не дифференцируема?

Очевидно, что результат макс-пулинга не изменится, если мы немного подвинем входные данные, не являющиеся максимальными. То есть при варьировании этих не максимальных элементов результат макс-пулинга не меняется, а значит, производная по этим элементам равна 0.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/28f987aa5ff3a0d59d25563faee9e35f/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl33_1.png)

При изменении максимального элемента максимум сразу меняется, и производная по этому элементу равна 1.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/946411e05d3b76f14992301697f61089/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl33_2.png)

Таким образом мы можем посчитать градиент пулинга.

### Подведём итоги

* Пулинг применяется с бόльшим stride и обеспечивает более быстрое увеличение поля обзора (экспоненциальный рост против линейного).
* Взятие максимума вместо обучаемой свёртки экономит вычисления и добавляет инвариантность к сдвигу.

# 9. Практика. Пулинг слой

Используемый в видео [ноутбук](https://colab.research.google.com/drive/1Wteyx4m3mbwnzkXBFnLRQo7Fwmc_Kl51).

Попробуем добавить в нашу сеть пулинг слой. Мы теперь знаем, что он нужен для того, чтобы **поле обзора росло быстрее**, а также для того, чтобы **появлялась небольшая инвариантность к сдвигу**. 

Выберем из списка слой MaxPool2D, так как будем делать пулинг по окошкам 2 × 2.

```py
model.add(L.Conv2D(filters=32, kernel_size=3, strides=1, padding='same', input_shape=(28, 28, 1)))
model.add(L.MaxPool2D())
model.add(L.Conv2D(filters=64, kernel_size=3, strides=1, padding='same'))
```
Заметим, что слой MaxPool2D в два раза уменьшил размер нашей картинки, количество фильтров не изменилось.

При этом мы использовали 0 параметров.

# 10. Первая свёрточная сеть

### Соберём это всё в свёрточную сеть

Теперь мы готовы собрать нашу первую свёрточную сеть. 

Для примера рассмотрим архитектуру LeNet, придуманную в 1998 году. Она применялась для MNIST — задачи распознавания рукописных цифр.

На вход принималась чёрно-белая картинка размера 32 × 32 × 1. Сначала предлагается применить свёртку с фильтром размером 5 × 5 и обучить шесть таких фильтров, чтобы на выходе получить объём 28 × 28 × 6 (32 превратилось в 28 потому, что мы не использовали padding). Дальше предлагается использовать пулинг слой, чтобы уменьшить картинку в 2 раза, таким образом получив размеры 14 × 14 × 6. 

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/1a51603a2c5e47854ae3a019208bae54/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl36.png)

Количество фильтров при этом не поменяется, так как пулинг работает с каждой feature map независимо.

Потом применяем ещё один свёрточный слой с фильтром 5 × 5, чтобы увеличить поле обзора, и получаем размеры 10 × 10 × 16. Фильтров, которые обучаются, становится уже больше. Это делается для того, чтобы не терять информацию. 

Вновь используем пулинг слой, который сделает нашу картинку размером 5 × 5 × 16. Полученный объём предлагается вытянуть в вектор и к этому вектору применить 2 полносвязных слоя, то есть предполагается, что в этом последнем пулинг слое в 16 признаках уже закодирована нужная информация, которая поможет нам воссоздать цифру.

После применения двух полносвязных слоёв нужен выходной слой, на котором будет десять выходов, так как у нас десять цифр, и будет применена функция softmax, которая превратит любые выходы в правильное распределение вероятностей. 

Если взять 60 000 примеров, где у нас есть картинка на входе и для неё известен класс, собрать эту архитектуру и оставить backpropagation на пару минут, то он выучит все веса на всех слоях, которые решают эту задачу практически идеально.

### Как работает функция softmax?

Допустим, мы посчитали десять выходов. Каждый из них заменим на экспоненту в степени этого выхода и пронормируем все полученные экспоненты, чтобы в сумме они давали 1. Так как экспонента неотрицательна, а их сумма равна 1, то получим правильное распределение вероятностей.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/630df4984c912617e14901c73ad1b8f2/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl37.png)


### Кросс-энтропия

Чтобы запустить backpropagation, нужно знать, какую функцию потерь использовать. Для классификации на K классов обычно используют функцию потерь, которую называют **кросс-энтропия** (cross-entropy).

Для каждого примера мы идём по всем возможным классам и, если пример принадлежит этому классу, то мы в потери записываем логарифм со знаком «минус» от предсказанной вероятности этого класса.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/d7c1e1cd2c30e7f8b4ba23e0323db679/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl38.png)

По графику функции -ln(х) видно, что если аргумент этой функции близок к 1, то потери близки к 0, а если аргумент близок к 0, то потери уходят в бесконечность. Это означает, что в потерях на правильном классе хочется видеть вероятность как можно более близкую к 1. Это и будет решать нашу задачу  классификации.

Если обучить нашу нейросеть и посмотреть на каждом слое изображения, на какие кусочки реагируют (имеют большую активацию) наши нейроны, то заметим, что на первом слое нейроны реагируют на простые паттерны на изображении (например, они могут находить перепады яркости под разными углами). Второй свёрточный слой уже может распознавать что-то более сложное (например, кусочек глаза) и так далее. Таким образом нейросеть учит иерархические шаблоны.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/e231859d182ac7c08b0db1f905809ca2/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl39.png)

# 11. Практика. Первая свёрточная сеть

Используемый в видео [ноутбук](https://colab.research.google.com/drive/1Wteyx4m3mbwnzkXBFnLRQo7Fwmc_Kl51).

Теперь, используя имеющиеся знания, дособерём сеть для решения нашей задачи распознавания одежды по картинке в Fashion MNIST.

Добавим ещё один свёрточный слой и ещё один пулинг-слой, чтобы уменьшить размер картинки:

```py
K.clear_session()

model = M.Sequential()
model.add(L.Conv2D(16, kernel_size=3, strides=1, padding='same', input_shape=(28, 28, 1)))
model.add(L.MaxPool2D())
model.add(L.Conv2D(32, kernel_size=3, strides=1, padding='same'))
model.add(L.MaxPool2D())
model.add(L.Conv2D(64, kernel_size=3, strides=1, padding='same'))
model.add(L.MaxPool2D())
```

Теперь, когда картинка стала размера 3 × 3 × 64, вытянем все имеющиеся нейроны в один плоский вектор. Поверх этого плоского вектора мы можем наслоить пару полносвязных слоёв  и решить нашу задачу классификации.

Для того чтобы вытянуть картинку в вектор, добавляем слой Flatten:

```py
model.add(L.Flatten()) 
```
На выходе получим:      ```flatten_1 (Flatten) (None, 576) 0```

576 — это ровно та размерность, которая у нас была: 3 × 3 × 64 = 576.

Теперь нам понятно, как работать с этим вектором. К нему можно применить полносвязные слои и получить новый ветор на выходе. Применим плотный слой, у которого будет десять выходов (так как у нас десять классов) с функцией активации softmax, поскольку она даёт распределение вероятностей.

```model.add(L.Dense(10, activation='softmax'))```

На выходе получим сеть с 30 000 параметров, а 576 нейронов превратились в десять.

Мы готовы к обучению. Но, чтобы получить больше параметров, добавим ещё один плотный слой перед крайним:

```model.add(L.Dense(64, activation='elu'))```

Получим 60 874 параметров, что нас как раз устраивает.

Будем компилировать модель, минимизируя кросс-энтропию и используя оптимайзер Adam:

```py
model.compile(
    loss='categorical_crossentropy',  # минимизируем кросс-энтропию
    optimizer='adam',  # так будет быстрее, позже узнаем что это :)
    metrics=['accuracy']  # выводим процент правильных ответов
)
```
Далее нормируем и центрируем картинку (так сети будет проще учиться) и делаем fit, задав фиктивную размерность:

```x_train_float[:, :, :, np.newaxis]```

Заметим, что обучение идёт очень медленно, потому что мы считаем на центральном процессоре. Запустим наш процесс на GPU, сменив runtime type. После этого нас перекинут в новый контейнер, в котором есть видеокарта. Запустим все ячейки заново.

Заметим, что здесь уже на второй эпохе мы получаем качество 0,89, в то время как с полносвязными слоями получали 0,87. Наши свёрточные слои решают задачи с картинками лучше, что неудивительно.

Таким образом, мы собрали нашу первую свёрточную сеть.

# 12. Современные архитектуры

Вспомним про задачу ImageNet. Люди за много лет насобирали более миллиона изображений, которые размечены на 1 000 классов. Классы довольно сложные (например, разные породы собак и кошек). Для неподготовленного человека разделить эти изображения на классы достаточно сложно.

В 2012 году решению этой задачи обучили огромную **нейросеть AlexNet**.

Устроена она была так: у нас есть несколько свёрточных слоев с макс-пулингом между ними, и мы используем свёртки 11 × 11, 5 × 5, 3 × 3.

Дополнительно использовался dropout, аугментация данных и ReLu (специальная функция активации). Про них узнаем в следующем модуле, а сейчас посмотрим вот на что: в этой сети было 60 миллионов параметров, и это была беспрецедентная сеть на тот момент.

Про аугментацию данных — один из трюков нейросети AlexNet — можем поговорить уже сейчас.

В нейросети было 60 миллионов параметров, которые нужно оценить по миллиону изображений. На самом деле картинок слишком мало, чтобы оценить столько параметров. Попробуем слегка видоизменить картинки (повернуть, отразить, растянуть, поменять резкость) и тогда наши данные для обучения увеличатся. Все эти манипуляции с картинками и называются **аугментацией данных**.

Стоит отметить, что добавлять сдвиги при аугментации данных бесполезно.

Можно было бы предположить, что сдвиги помогут нам научиться искать котов на разных положениях на картинке, но нам это не нужно, если у нас есть макс-пулинг слой, ведь достаточно просто сделать архитектуру таким образом, чтобы где-то близко к выходному слою мы взяли максимум.

Другой пример архитектуры — **VGG**, предложенная в 2015 году (спустя 3 года после AlexNet). Видеокарты к тому времени стали мощнее, и новая сеть содержала 138 миллионов параметров. Здесь уже не использовали дорогие и тяжёлые свёртки размера 11 × 11, а обошлись большим числом свёрток размером 3 × 3. В остальном по принципу нейросеть очень похожа на AlexNet.

# 13. Inception V3

Теперь познакомимся с архитектурой **Inception V3**, предложенной также в 2015 году и отличающейся от AlexNet. В этой сети значительно меньше параметров (всего 25 миллионов). Эта сеть состоит из оригинальных inception-блоков (их рассмотрим чуть позже) и дополнительных трюков, таких как батч-нормализация, аугментация и RMSProp (алгоритм оптимизации).

### Как устроен inception-блок?

Первое, с чем нам нужно познакомиться, — свёртка 1 × 1.

Пусть у нас есть входной объём (светло-оранжевый на картинке). И если у нас есть окошко 1 × 1, это означает, что мы берём глубокий пиксель из $C_{in}$ объёма и хотим получить другой пиксель другой глубины.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/4ea7269ee3263b0c498f7aafa5281c08/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl51.png)

То есть мы будем искать новые пиксели как линейные комбинации входных. Такое линейное преобразование очень похоже на уменьшение размерности.

Объясняя простым языком, мы даём нейросети возможность обучать много фильтров, а затем похожие фильтры схлопываются в один линейным преобразованием.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/893d44386e5b52e7cf4858d231985ea0/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl52.png)

### Как устроен inception-блок внутри?

На входе нам поступает некоторый объём, и к нему применяют четыре ветки вычисления:

1. Первая ветка применяет свёртку 1 × 1, чтобы уменьшить глубину этого объёма. Дальше применяется свёртка 5 × 5 с таким padding, чтобы на выходе высота и ширина изображения остались неизменными.
1. На второй ветке применяем свёртку 1 × 1, а затем 3 × 3.
1. На третьей ветке применяем пулинг с шагом 1, а затем свёртку 1 × 1.
1. Четвёртая ветка просто делает свёртку 1 × 1.

Отметим, что можно так подобрать padding и stride, что измерения W и H выходных объёмов всех веток будут одинаковыми, а глубина будет различаться. Это означает, что их можно будет склеить по глубине. В итоге получим один большой объём.

Можно заменить свёртки 5 × 5 двумя свёртками 3 × 3:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/924ede84a0dd14b5311f092b5bbd9a96/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl53.png)

Количество параметров, необходимых для такой операции, станет меньше.

### Сепарабельные фильтры

Существует фильтр размытия по Гауссу — гауссиан, то есть мы возьмём кусочек изображения, взвесим все пиксели с весами, которые даёт нам гауссиан, и получим размытие по Гауссу. Это размытие можно «дёшево» сделать одномерными свёртками. 

Например, пусть у нас есть синий пиксель. Сначала сделаем размытие одномерным гауссовским ядром по горизонтали:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/5f20378a3e9760c56b8d068119b1356d/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl54.png)

К этому результату можно применить свёртку по вертикали, и мы получим такой же результат, как если бы мы применяли двумерную свёртку к нашему пикселю. Если посчитать количество операций при размытии по горизонтали, а затем по вертикали, то количество операций окажется в разы меньше.

Такие фильтры, которые позволяют заменить себя на последовательность одномерных, называются **сепарабельными**.

Дадим нейросети возможность учить такие фильтры, заменим в нашем inception-блоке свёртки 3 × 3 на сепарабельные свёртки 1 × 3 и 3 × 1.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/1a388a1d7cc97b8be75b0c72b94e3552/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/m3sl55.png)

Таким образом, мы придумали очень эффективный inception-блок, который использует мало параметров и даёт лучшее качество, чем набор свёрток 3 × 3.

# 14. Домашнее задание


В домашнем задании вам предстоит обучить свёрточную нейронную сеть распознавать изображения из датасета CIFAR-10. 

[Notebook](https://colab.research.google.com/drive/1Hs7xYVkjOO7s_b8_ozK9VVCKwelVe27O) с инструкциями по решению задачи. 

Чтобы завершить домашнюю работу, введите свои ответы на вопросы в поля ниже. 

> А также мы подготовили для вас Kaggle-соревнование по распознаванию CAPTCHA, пройдите по ссылке [здесь](https://www.kaggle.com/c/sf-captcha-recognition/).

Желаем удачи!

### Дополнительные материалы

Дополнительно мы предлагаем вам ознакомиться с интересными материалами, которые вы можете сохранить себе и изучить в удобное время.

* [Convolutional Networks](http://cs231n.github.io/convolutional-networks/)
* [How do Convolutional Neural Networks work?](https://brohrer.github.io/how_convolutional_neural_networks_work.html)
* [How convolutional neural networks see the world?](https://blog.keras.io/how-convolutional-neural-networks-see-the-world.html)